In [ ]:
#Install opendatasets library
#Python’s opendatasets library is used for downloading open datasets from platforms such as Kaggle.
!pip install opendatasets

In [ ]:
# Import the opendatasets library
import opendatasets as od

In [ ]:
#On executing the below line, it will prompt for Kaggle username. Kaggle username 
#can be fetched from the Account tab of the My Profile section
#On entering the username, it will prompt for Kaggle Key. Again, go to the account tab of the My Profile section and click on Create New API Token.
#This will download a kaggle.json file.
od.download("https://www.kaggle.com/datasets/imdevskp/corona-virus-report")

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# Load covid report data into a DataFrame
df = pd.read_csv("corona-virus-report/covid_19_clean_complete.csv")

# Filter and clean the DataFrame
df_transform = df[(df['Confirmed'] >1) &
                 (df['Deaths'] >= 0)]
#df_transform = df.dropna()
df_transform


In [7]:
#Create/connect to a SQLite database
conn = sqlite3.connect('covid.db')

In [8]:
create_sql = '''
    CREATE TABLE IF NOT EXISTS longlat (
        "Province/State" TEXT,
        "Country/Region" TEXT,
        Lat REAL,
        Long REAL,
        "WHO Region" TEXT
    )
'''
cursor = conn.cursor()
cursor.execute(create_sql)


In [10]:
create_sql = '''
    CREATE TABLE IF NOT EXISTS country (
        "Country/Region" TEXT,
        "Date" TEXT,
        "Confirmed" INTEGER, 
        "Deaths" INTEGER,
        "Recovered" INTEGER,
        "Active" INTEGER,
        "WHO Region" TEXT,
         PRIMARY KEY ("Country/Region")
    )
'''
cursor = conn.cursor()
cursor.execute(create_sql)

In [11]:
# Insert data into the "longlat" table
df_transform[['Province/State', 'Country/Region', 'Lat', 'Long', 'WHO Region']].to_sql('longlat', conn, index=False, if_exists='replace')

# Insert data into the "country" table
df_transform[['Province/State', 'Country/Region', 'Date', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'WHO Region']].to_sql('country', conn, index=False, if_exists='replace')

37800

In [12]:
# import sqlite3

# # Replace 'your_database.db' with the actual name of your SQLite database file
# conn = sqlite3.connect('your_database.db')
# Replace 'your_table' with the actual name of the table you want to check
table_name = 'longlat'

# Fetch data from the table
query = f"SELECT * FROM {table_name}"
existing_data = pd.read_sql_query(query, conn)

# Print or examine the existing data
print(existing_data)

      Province/State         Country/Region        Lat        Long  \
0            Beijing                  China  40.182400  116.414200   
1          Chongqing                  China  30.057200  107.874000   
2          Guangdong                  China  23.341700  113.424400   
3            Guangxi                  China  23.829800  108.788100   
4             Hainan                  China  19.195900  109.745300   
...              ...                    ...        ...         ...   
37795           None  Sao Tome and Principe   0.186400    6.613100   
37796           None                  Yemen  15.552727   48.516388   
37797           None                Comoros -11.645500   43.333300   
37798           None             Tajikistan  38.861000   71.276100   
37799           None                Lesotho -29.610000   28.233600   

                  WHO Region  
0            Western Pacific  
1            Western Pacific  
2            Western Pacific  
3            Western Pacific  
4   

In [12]:
#close connection
conn.commit()
conn.close()